In [1]:
from bs4 import BeautifulSoup
from random import randint, shuffle
from time import sleep
import re
import pandas as pd
import json
import math
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
from tqdm import tqdm_notebook as tqdm
import shutil
import numpy as np
import pandas_profiling
import pickle

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", -1)
pd.set_option("display.float_format", "{:,.4f}".format)
pd.set_option("display.max_info_rows", 200)


import surprise
from surprise.model_selection import train_test_split
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD, SVDpp, BaselineOnly, NMF, CoClustering
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white")
sns.set_context("paper", font_scale=1.8)
sns.set_palette("hls", 15)

# Dataset

In [2]:
# List of all products from Sephora as of June 24 with binary columns for categories

products = pd.read_csv("df_sephora.csv")
products.head(1)

,id,name,brand_id,brand_name,short_desc,long_desc,item,item_name,list_price,variation,variants,price_low,price_high,is_limited_edition,rating,review_count,user_favorites,categories,url,image_url,ingredients,suggested_usage,brand_long,has_variants,has_categories,has_ingredients,has_suggested_usage,is_rated,cologne,men,fragrance,perfume,women,lotions_and_oils,rollerballs_and_travel_size,perfume_gift_sets,value_and_gift_sets,lipstick,lip,makeup,lip_balms_and_treatments,lip_treatments,mascara,eye,lip_gloss,eyebrow,body_sprays_and_deodorant,bath_and_shower,cologne_gift_sets,body_mist_and_hair_mist,concealer,face,lip_stain,deodorant_and_antiperspirant,body_care,bath_and_body,face_primer,foundation,tinted_moisturizer,face_serums,treatments,skincare,mists_and_essences,moisturizers,highlighter,liquid_lipstick,lip_sets,face_sets,setting_spray_and_powder,aftershave,shaving,eye_creams_and_treatments,eye_care,face_masks,masks,face_wash_and_cleansers,cleansers,exfoliators,makeup_removers,eyeliner,face_brushes,makeup_brushes_and_applicators,makeup_palettes,eye_palettes,bb_and_cc_creams,blush,cheek,facial_peels,eyelash_curlers,accessories,eye_sets,mini_size,bronzer,face_oils,contour,lip_liner,eyeshadow,face_wash,moisturizer_and_treatments,toners,body_lotions_and_body_oils,body_moisturizers,scrub_and_exfoliants,self_tanners,face_sunscreen,sun_care,blemish_and_acne_treatments,facial_cleansing_brushes,high_tech_tools,for_face,color_correct,skincare_sets,decollete_and_neck_creams,hand_cream_and_foot_cream,candles_and_home_scents,night_creams,sheet_masks,eye_brushes,beauty_supplements,wellness,facial_rollers,body_sunscreen,cellulite_and_stretch_marks,for_body,body_wash_and_shower_gel,nail,other_needs,shampoo_and_conditioner,hair,hair_styling_and_treatments,conditioner,shampoo,eye_masks,eye_primer,mirrors_and_sharpeners,tools_and_brushes,cheek_palettes,face_wipes,tweezers_and_eyebrow_tools,hair_accessories,hair_tools,gift_cards,false_eyelashes,brush_sets,brush_cleaners,makeup_and_travel_cases,hair_removal,lip_brushes,sponges_and_applicators,makeup_bags_and_travel_cases,hair_masks,curling_irons,hair_straighteners_and_flat_irons,hair_dryers,dry_shampoo,hair_brushes_and_combs,scalp_and_hair_treatments,bath_soaks_and_bubble_bath,hair_styling_products,blotting_papers,hair_primers,hair_oil,teeth_whitening,anti_aging,travel_size,facial_men,candles,diffusers,holistic_wellness,leave_in_conditioner,hair_spray,color_care,best_for,just_arrived,gifts_her,gifts_men,gifts_them,gifts_teens,gift_sets_men,gift_sets,editors_picks,luxury_gifts,sephora_exclusives,perfume_men,grooming_men,hair_men,personal_care_men,just_arrived_men,best_seller_men,vegan,cruelty_free,for_men
0,P180726,Pour Homme II,1063,Gucci,"An alluring fragrance heightened by a sensual, spicy soul, Gucci Pour Homme II is the essence of young, masculine sensuality.","An alluring fragrance heightened by a sensual, spicy soul, Gucci Pour Homme II is the essence of young, masculine sensuality. The embodiment of relaxed refinement for the modern-minded man whose style is distinctive and effortless, this fresh, spicy, woody, scent is filled with playful attitude and charisma. Notes: Bergamot, Violet Leaves, Cinnamon, Pimento, Black Tea, Myrrh, Tobacco Leaves, White Musk, Olive Wood. Style: Seductive. Masculine. Free-spirited.",997338,3.3 oz/ 98 mL Eau de Toilette Spray,92.0000,Size + Concentration + Formulation,NaN,92.0000,92.0000,0,4.8070,114,2030,"{'cat60150': 'Cologne', 'cat1230040': 'Men', 'cat160006': 'Fragrance'}",https://www.sephora.com/product/pour-homme-ii-P180726?skuId=997338,https://www.sephora.com/productimages/sku/s997338-main-zoom.jpg,"Alcohol Denat., Fragrance, Water, Ethylhexyl Methoxycinnamate, Diethylamino Hydroxybenzoyl Hexyl Benzoate, BHT, Benzyl Salicylate, Linalool, Butylphenyl Methylpropional, Hydroxycitronellal, Limonene, Hexyl Cinnamal, Citronellol, Alpha-Isomethyl Ionone, Geraniol, Hydroxyisohexyl 3-Cyclohexene Carboxaldehyde, Citral, Farnesol, Isoeugenol, Anise Alcohol, Benzyl Ben

In [239]:
products.info(null_counts=True, max_cols=200)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9007 entries, 0 to 9006
Data columns (total 193 columns):
id                                   9007 non-null object
name                                 9007 non-null object
brand_id                             9007 non-null int64
brand_name                           9007 non-null object
short_desc                           8997 non-null object
long_desc                            8997 non-null object
item                                 9007 non-null int64
item_name                            7696 non-null object
list_price                           9007 non-null float64
variation                            9007 non-null object
variants                             3930 non-null object
price_low                            9007 non-null float64
price_high                           9007 non-null float64
is_limited_edition                   9007 non-null int64
is_natural_organic                   9007 non-null int64
rating                 

In [154]:
# Product reviews for the past 2 years where users recommended and rated the product 3 or above.

product_reviews = pd.read_csv("selected_reviews_short2.csv")
product_reviews

/Applications/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,product_id,author_id,date,review_id,rating,helpfulness,feedback_count,positive_feedback,negative_feedback,comment_count,review_title,review_text,skintype,eyecolor,skinconcerns,staffcontext,haircolor,skintone,incentivizedreview,verifiedpurchaser,age,hairconcerns,haircondition
0,P0417,5869746961,2019-09-14 17:16:59+00:00,135835739,3,0.67,3,2,1,0,NaN,"It doesn't help with blackheads at all, but it does with whiteheads. It is okay",combination,green,acne,False,brunette,light,False,NaN,NaN,NaN,NaN
1,P0417,22189528930,2019-07-07 17:48:13+00:00,130283148,3,1.00,5,5,0,0,NaN,"Be careful using this, it does extract pimples but only if they are at a head and ready to pop. Otherwise the harder you push to bring it out marks your skin. It’s better than using your nails and bruising your skin on stubborn pimples but still can damage your skin. The black head extractor does work fantastic, brings out the really stubborn black heads but then again also damages your skin if your press too hard. Trail and error learning how hard to press before making marks.",combination,blue,acne,False,black,medium,False,NaN,NaN,NaN,NaN
2,P0417,5641704837,2018-09-15 16:34:33+00:00,118730320,3,2.00,0,0,0,0,NaN,It works but I think it’s not as gentle as I would like. It can still cause trauma on your skin. So be gentle! Better than using your fingers.,combination,blue,acne,NaN,red,fair,NaN,NaN,NaN,NaN,NaN
3,P04546871,22081371793,2020-05-17 18:17:18+00:00,156485522,3,1.00,3,3,0,0,"She’s cute, glitter will get on your clothing",Bought this on sale and used it for a shoot at home. It’s super cute and feels very nice with amazing sparkle pay off in the light. Highly recommend using a body brush for this just to avoid it being clustered in your hands. The glitter will get on everything you touch your skin on as a heads up. Great for warm outings to amp your look.,combination,brown,acne,False,brunette,medium,False,NaN,NaN,NaN,NaN
4,P04546871,1440172327,2019-11-12 01:50:25+00:00,138255643,3,2.00,0,0,0,0,Beautiful but Sticky,The pay off the glitter is absolutely beautiful but it’s sticky. That is why I didn’t give this 4 stars.,combination,brown,aging,False,blonde,light,False,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899987,P99902,6041243828,2019-04-26 01:14:25+00:00,127314904,5,2.00,0,0,0,0,NaN,Love this Eyeliner. Stays in place,oily,hazel,NaN,False,brunette,light,False,NaN,NaN,NaN,NaN
899988,P99902,1334062044,2019-04-26 00:31:45+00:00,127313503,5,1.00,2,2,0,0,The only eyeliner you should own,"If you want a Truly Good Eyeliner buy this. Here is why I will only buy this brand and type of eyeliner 1. It doesnt irritate my eyes and cause eye boogers when using it on my waterline 2. It does NoT run. 3. It glides on without having to tug 4. It comes with a smudger at the end 5. It goes on properly the first time and if you want it extra thick, you can put it on another layer without causing that weird skipping effect. Yes it is absurdly expensive, but I dont have time for drama and problems when I am getting ready for work. It is worth it. Get it, but only get this one.",combination,NaN,NaN,False,black,olive,False,NaN,NaN,NaN,NaN
899989,P99902,5649540,2018-10-13 05:25:38+00:00,120021816,5,1.00,14,14,0,0,A must for an eyeliner that STAYS on!!!,"I am so highly impressed at this waterproof liner that I had to post about it.\nI lost my son in April and well, I cry a lot. I haven't been out much lately when when I do go out I use this liner since it is truly indelible like ink! I remove it with an oil based remover for bet removal. I get complements [I'm older too]. The Intense Brown and Captivating Blue is quite beautiful on eyes even over the black or brown..unlike all my others these just work and look best. I have a purple/plum shade not shown here plus more not seen on here but in teh stores are gorgeous on. There is a short smudge time after lining but it's so worth it. I own maybe 50 liners lol mid to 

In [155]:
product_reviews.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899992 entries, 0 to 899991
Data columns (total 23 columns):
product_id            899992 non-null object
author_id             899992 non-null object
date                  899992 non-null object
review_id             899992 non-null int64
rating                899992 non-null int64
helpfulness           899992 non-null float64
feedback_count        899992 non-null int64
positive_feedback     899992 non-null int64
negative_feedback     899992 non-null int64
comment_count         899992 non-null int64
review_title          600845 non-null object
review_text           895058 non-null object
skintype              888914 non-null object
eyecolor              872991 non-null object
skinconcerns          523567 non-null object
staffcontext          623709 non-null object
haircolor             882991 non-null object
skintone              888803 non-null object
incentivizedreview    623709 non-null object
verifiedpurchaser     233798 non-null o

In [152]:
# product_reviews = product_reviews[product_reviews.helpfulness>=0.5].copy()

In [153]:
# product_reviews.to_csv("selected_reviews_short2.csv", index=False)

# Recommender System

In [14]:
df = pd.read_csv("user_item_rating_raw.csv")
df = df[["author_id", "uid", "product_id", "rating"]].copy()
df

,author_id,uid,product_id,rating
0,23023065402,0_0_0_0_0_0_0_0,P122735,3
1,1886707486,0_0_0_0_0_0_0_0,P12573,3
2,5372680671,0_0_0_0_0_0_0_0,P128706,3
3,10612140468,0_0_0_0_0_0_0_0,P163103,3
4,5742906597,0_0_0_0_0_0_0_0,P174502,3
...,...,...,...,...
885187,2489107310,over54_tan_oily_darkCircles_brown_black_0_0,P440065,5
885188,26041356151,over54_tan_oily_sunDamage_brown_blonde_0_0,P426340,5
885189,26041356151,over54_tan_oily_sunDamage_brown_blonde_0_0,P454033,5
885190,26041356151,over54_tan_oily_sunDamage_brown_blonde_0_Volumizing,P433813,5


In [8]:
author_ids = list(df.uid.unique())
product_ids = list(df.product_id.unique())

In [10]:
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(3, 5))

In [11]:
# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[["uid", "product_id", "rating"]], reader)

In [12]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  34749 

Number of items:  6839


In [13]:
# del df

In [8]:
trainset, testset = train_test_split(data, random_state=42)

In [9]:
knn_base = KNNBaseline(verbose=True)
knn_base.fit(trainset)
accuracy.rmse(knn_base.test(testset)), accuracy.mae(knn_base.test(testset))

# RMSE: 0.5341
# MAE:  0.3991

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.5341
MAE:  0.3991


(0.5341034194390785, 0.39906235232037407)

In [314]:
knn_basic = KNNBasic(verbose=True)
knn_basic.fit(trainset)
accuracy.rmse(knn_basic.test(testset)), accuracy.mae(knn_basic.test(testset))

# RMSE: 0.5366
# MAE:  0.3927

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.5366
MAE:  0.3927


(0.5365941508492791, 0.39273999839571455)

In [10]:
benchmark = []
# Iterate over all algorithms
for algorithm in tqdm([KNNBaseline(), KNNBasic()]):
    # Perform cross validation
    results = cross_validate(algorithm,
                             data,
                             measures=['RMSE', 'MAE'],
                             cv=5,
                             verbose=False)

    # Get results & append algorithm name
    results_table = pd.DataFrame.from_dict(results).mean(axis=0)
    results_table = results_table.append(
        pd.Series([str(algorithm).split(' ')[0].split('.')[-1]],
                  index=['Algorithm']))
    benchmark.append(results_table)

cv_results = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.



In [11]:
cv_results

,test_rmse,test_mae,fit_time,test_time
Algorithm,,,,
KNNBaseline,0.5335,0.3986,85.5507,100.4008
KNNBasic,0.5360,0.3923,87.6866,94.8088


In [1]:
params = {
    'n_factors': [50, 100],
    'reg_all': [0.02, 0.05, 0.1],
    'lr_all': [0.001, 0.01]
}
gs_KNN = GridSearchCV(KNNBaseline, param_grid=params, n_jobs=1, cv=5,  joblib_verbose=1)

In [2]:
# gs_SVD.fit(data)

In [3]:
# pd.DataFrame(gs_SVD.cv_results)[[
#     "rank_test_rmse", "mean_test_rmse", "std_test_rmse", "rank_test_mae",
#     "mean_test_mae", "std_test_mae", "mean_fit_time", "mean_test_time",
#     "params"
# ]].sort_values("rank_test_rmse")

In [1]:
# pickle.dump(algo, open( .pkl", "wb"))

In [8]:
# algo = pickle.load(open("SVD_best.pkl", "rb"))

In [39]:
def age_range(age):

    age_dict = {
        "13to17": range(1, 17 + 1),
        "18to24": range(18, 24 + 1),
        "25to34": range(25, 34 + 1),
        "35to44": range(35, 44 + 1),
        "45to54": range(45, 54 + 1),
        "over54": range(54, 100 + 1),
        "0": [0]
    }

    if age > 100:
        print("please check age input")
        return

    for key, val in age_dict.items():
        if age in val:
            age_group = key
            if age < 13:
                print("products may not be aged appropriate")
            return age_group

In [40]:
age_range(12)

products may not be aged appropriate


'13to17'

In [69]:
product_ids

['P122735',
 'P12573',
 'P128706',
 'P163103',
 'P174502',
 'P17876556',
 'P186805',
 'P195321',
 'P202633',
 'P228914',
 'P245205',
 'P255506',
 'P263504',
 'P269122',
 'P270539',
 'P293605',
 'P309310',
 'P313020',
 'P376136',
 'P377485',
 'P377724',
 'P378017',
 'P378111',
 'P378145',
 'P379509',
 'P380541',
 'P381000',
 'P381176',
 'P384467',
 'P384954',
 'P386464',
 'P386606',
 'P388615',
 'P390932',
 'P391306',
 'P393661',
 'P394639',
 'P395105',
 'P396012',
 'P396733',
 'P397401',
 'P397517',
 'P39787544',
 'P398001',
 'P398009',
 'P399623',
 'P400207',
 'P400252',
 'P400259',
 'P401463',
 'P4015',
 'P4016',
 'P402071',
 'P403800',
 'P403806',
 'P404758',
 'P404820',
 'P404830',
 'P405102',
 'P406080',
 'P406104',
 'P407365',
 'P408219',
 'P409847',
 'P4103',
 'P411031',
 'P411387',
 'P412093',
 'P412095',
 'P412392',
 'P413039',
 'P414737',
 'P415438',
 'P415624',
 'P415763',
 'P415771',
 'P416366',
 'P417141',
 'P417626',
 'P418218',
 'P419022',
 'P419219',
 'P419506',
 'P4202

In [78]:
age = 36 
ager = age_range(age)
stone = "medium" # 'light' 'medium' 'fair' 'ebony' 'olive' 'deep' 'dark' 'tan' 'porcelain'
stype = "0" # 'combination' 'oily' 'normal' 'dry'
sconcerns = "0" # 'acne' 'aging' 'blackheads' 'unevenSkinTones' 'sensitivity' 
# 'darkCircles' 'calluses' 'pores' 'redness' 'dullness' 'cellulite'
# 'sunDamage' 'cuticles' 'stretchMarks' 'puffiness'
ecolor = "brown" # 'green' 'blue' 'brown' 'hazel' 'gray'
hcolor = "0" # brunette' 'black' 'red' 'blonde' 'gray' 'auburn'
hcondition = "0" #'chemicallyTreated' 'normal' 'wavy' 'fine' 'curly' 'coarse' 'dry'
# 'oily' 'straight'
hconcerns = "0"# 'ColorProtection' 'Damage' 'CurlyEnhancing' 'Dandruff' 'Volumizing'
#  'AntiAging' 'Frizz' 'Hold' 'Thinning' 'Oiliness' 'StraighteningSmoothing'
#  'HeatProtection' 'Shine'

budget_low = 50
budget_max = 100

uid_query=ager+"_"+stone+"_"+stype+"_"+sconcerns+"_"+ecolor+"_"+hcolor+"_"+hcondition+"_"+hconcerns.title()


items_to_pred = [[uid_query, iid, 0] for iid in product_ids]
predictions = algo.test(items_to_pred)
predictions

[Prediction(uid='35to44_medium_0_0_brown_0_0_0', iid='P17876556', r_ui=0, est=4.7335473163304504, details={'was_impossible': False})]

In [79]:
items_to_pred = [[uid_query, iid, np.nan] for iid in product_ids]
predictions = algo.test(items_to_pred)
pred_ratings = np.array([pred.est for pred in predictions])

# map rating preds to product id
top_products = pd.DataFrame(data=pred_ratings,
                            columns=["pred_ratings"],
                            index=product_ids).sort_values(
                                "pred_ratings",
                                ascending=False).reset_index().head(500)

# get product details
top_products = pd.merge(top_products,
                        products,
                        how="left",
                        left_on="index",
                        right_on="id")

del items_to_pred, pred_ratings, predictions

top_products = top_products[(top_products.price_low >= budget_low)
                            & (top_products.price_high <= budget_max)]
top_products.sort_values(
    ["pred_ratings", "rating", "review_count"], ascending=False).head(15).drop(
        columns=["long_desc", "brand_long", "ingredients", "suggested_usage"])

,index,pred_ratings,id,name,brand_id,brand_name,short_desc,item,item_name,list_price,variation,variants,price_low,price_high,is_limited_edition,rating,review_count,user_favorites,categories,url,image_url,has_variants,has_categories,has_ingredients,has_suggested_usage,is_rated,cologne,men,fragrance,perfume,women,lotions_and_oils,rollerballs_and_travel_size,perfume_gift_sets,value_and_gift_sets,lipstick,lip,makeup,lip_balms_and_treatments,lip_treatments,mascara,eye,lip_gloss,eyebrow,body_sprays_and_deodorant,bath_and_shower,cologne_gift_sets,body_mist_and_hair_mist,concealer,face,lip_stain,deodorant_and_antiperspirant,body_care,bath_and_body,face_primer,foundation,tinted_moisturizer,face_serums,treatments,skincare,mists_and_essences,moisturizers,highlighter,liquid_lipstick,lip_sets,face_sets,setting_spray_and_powder,aftershave,shaving,eye_creams_and_treatments,eye_care,face_masks,masks,face_wash_and_cleansers,cleansers,exfoliators,makeup_removers,eyeliner,face_brushes,makeup_brushes_and_applicators,makeup_palettes,eye_palettes,bb_and_cc_creams,blush,cheek,facial_peels,eyelash_curlers,accessories,eye_sets,mini_size,bronzer,face_oils,contour,lip_liner,eyeshadow,face_wash,moisturizer_and_treatments,toners,body_lotions_and_body_oils,body_moisturizers,scrub_and_exfoliants,self_tanners,face_sunscreen,sun_care,blemish_and_acne_treatments,facial_cleansing_brushes,high_tech_tools,for_face,color_correct,skincare_sets,decollete_and_neck_creams,hand_cream_and_foot_cream,candles_and_home_scents,night_creams,sheet_masks,eye_brushes,beauty_supplements,wellness,facial_rollers,body_sunscreen,cellulite_and_stretch_marks,for_body,body_wash_and_shower_gel,nail,other_needs,shampoo_and_conditioner,hair,hair_styling_and_treatments,conditioner,shampoo,eye_masks,eye_primer,mirrors_and_sharpeners,tools_and_brushes,cheek_palettes,face_wipes,tweezers_and_eyebrow_tools,hair_accessories,hair_tools,gift_cards,false_eyelashes,brush_sets,brush_cleaners,makeup_and_travel_cases,hair_removal,lip_brushes,sponges_and_applicators,makeup_bags_and_travel_cases,hair_masks,curling_irons,hair_straighteners_and_flat_irons,hair_dryers,dry_shampoo,hair_brushes_and_combs,scalp_and_hair_treatments,bath_soaks_and_bubble_bath,hair_styling_products,blotting_papers,hair_primers,hair_oil,teeth_whitening,anti_aging,travel_size,facial_men,candles,diffusers,holistic_wellness,leave_in_conditioner,hair_spray,color_care,best_for,just_arrived,gifts_her,gifts_men,gifts_them,gifts_teens,gift_sets_men,gift_sets,editors_picks,luxury_gifts,sephora_exclusives,perfume_men,grooming_men,hair_men,personal_care_men,just_arrived_men,best_seller_men,vegan,cruelty_free,for_men
0,P459127,4.9411,P459127,Platinum Lip Plump SPF 30,6318,Dr. Lara Devgan Scientific Beauty,A lip plumper that improve the smoothness and hydration of the lips in the form of a clear gloss.,2374049,0.10 oz/ 3 mL,50.0000,None,NaN,50.0000,50.0000,0,4.9109,505,2652,"{'cat60109': 'Lip Balms & Treatments', 'cat1520035': 'Lip Treatments', 'cat150006': 'Skincare'}",https://www.sephora.com/product/dr-lara-devgan-scientific-beauty-platinum-lip-plump-P459127?skuId=2374049,https://www.sephora.com/productimages/sku/s2374049-main-zoom.jpg,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
1,P444401,4.9341,P444401,Solstice Eyeshadow Palette,6276,Aether Beauty,"A palette of 12 golden hour shades in matte, shimmer, satin, metallic, and duo-chrome finishes to have your eyes glowing like a sunset.",2189975,NaN,58.0000,None,NaN,58.0000,58.0000,0,4.9091,143,8835,"{'cat60045': 'Eye Palettes', 'cat130054': 'Eye', 'cat140006': 'Makeup'}",https://www.sephora.com/product/summer-solstice-eyeshadow-palette-P444401?skuId=2189975,https://www.sephora.com/productimages/sku/s218997

In [26]:
# for i in cv_knn_baseline.items():
#     print(i)

# np.mean(cv_knn_baseline['test_rmse'])

('test_rmse', array([0.63801205, 0.67060225, 0.68617023, 0.58904827, 0.66799372]))
('test_mae', array([0.52375997, 0.55111163, 0.54940623, 0.50025351, 0.55021826]))
('fit_time', (0.05592989921569824, 0.0341486930847168, 0.048558712005615234, 0.03869795799255371, 0.033309221267700195))
('test_time', (0.0018079280853271484, 0.0012211799621582031, 0.0014290809631347656, 0.001332998275756836, 0.0012969970703125))


0.6503653046223841